<img style="float: center;" src='https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png' alt="stsci_logo" width="900px"/> 

<a id="title_ID"></a>
# MIRI MRS Batch Processing Notebook #

**Author**: David Law, AURA Associate Astronomer, MIRI branch
<br>
**Last Updated**: June 29, 2023
<br>
**Pipeline Version**: 1.11.0

Customized by Michael Reefe, October 18, 2023

The purpose of this notebook is to provide a framework for batch processing of MIRI MRS data through all three pipeline stages.  Data is assumed to be located in two Observation folders (science and background) according to paths set up below.

This setup has been modified to produce the best quailty output data products that we have found for the observations of the Phoenix Cluster.  This includes enabling strict cosmic ray flagging, residual fringe correction, and 2D background subtraction, in addition to extra custom routines for flagging bad pixels and subtracting stirpe artifacts from cosmic rays.

Changes:<br>
Sep 1 2022: Add some commentary and example on how to use multicore processing in Detector 1<br>
Sep 12 2022: Disable unnecessary cube/1d spectra production for individual science exposures in Spec 2<br>
Oct 14 2022: Include residual fringe correction in spec2 (note that this will CRASH earlier pipeline versions!<br>
Jun 29 2023: Update to latest 1.11.0 pipeline with photom, outlier detection, and x1d changes, add CRDS path options.  Change to SMP LMC 058 demo.

<hr style="border:1px solid gray"> </hr>

1.<font color='white'>-</font>Configuration <a class="anchor" id="intro"></a>
------------------

In [ ]:
# Set parameters to be changed here.
# It should not be necessary to edit cells below this in general unless modifying pipeline processing steps.

import sys,os, pdb

# CRDS context (if overriding)
#%env CRDS_CONTEXT jwst_1093.pmap

# Set CRDS paths if not set already in your .bashrc shell configuration
#os.environ['CRDS_PATH']='/Users/dlaw/crds_cache'
#os.environ['CRDS_SERVER_URL']='https://jwst-crds.stsci.edu'
# Echo CRDS path in use
print('CRDS local filepath:',os.environ['CRDS_PATH'])
print('CRDS file server:',os.environ['CRDS_SERVER_URL'])

# Point to where the uncalibrated FITS files are from the science observation
input_dir = 'Phoenix_uncal/'

# Point to where you want the output science results to go
output_dir = 'Phoenix_cal/'

# Point to where the uncalibrated FITS files are from the background observation
# If no background observation, leave this blank
input_bgdir = 'Phoenix_uncal_bg/'

# Point to where the output background observations should go
# If no background observation, leave this blank
# output_bgdir = ''
output_bgdir = 'Phoenix_cal_bg/'

# Whether or not to process only data from a given band/channel
# Useful if overriding reference files
# Note BOTH must be set in order to work
use_ch='' # '12' or '34'
use_band='' # 'SHORT', 'MEDIUM', or 'LONG'

# Whether or not to run a given pipeline stage
# Science and background are processed independently through det1+spec2, and jointly in spec3

# Science processing
dodet1=True
dospec2=True
dospec3=True

# Background processing
dodet1bg=True
dospec2bg=True

# If there is no background folder, ensure we don't try to process it
if (input_bgdir == ''):
    dodet1bg=False
    dospec2bg=False

<hr style="border:1px solid gray"> </hr>

2.<font color='white'>-</font>Imports and setup <a class="anchor" id="intro"></a>
------------------

In [ ]:
# Now let's use the entire available screen width for the notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
# Basic system utilities for interacting with files
import glob
import time
import shutil
import warnings
import zipfile
import urllib.request

# Astropy utilities for opening FITS and ASCII files
from astropy.io import fits
from astropy.io import ascii
from astropy.utils.data import download_file
# Astropy utilities for making plots
from astropy.visualization import (LinearStretch, LogStretch, ImageNormalize, ZScaleInterval)

# Numpy for doing calculations
import numpy as np

# Matplotlib for making plots
import matplotlib.pyplot as plt
from matplotlib import rc

In [4]:
# Import the base JWST package
import jwst

In [ ]:
# JWST pipelines (encompassing many steps)
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline

# JWST pipeline utilities
from jwst import datamodels # JWST datamodels
from jwst.associations import asn_from_list as afl # Tools for creating association files
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase # Definition of a Lvl2 association file
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base # Definition of a Lvl3 association file

from stcal import dqflags # Utilities for working with the data quality (DQ) arrays
import miricoord
import miricoord.mrs.mrs_tools as mrstools

In [6]:
# Output subdirectories to keep science data products organized
# Note that the pipeline might complain about this as it is intended to work with everything in a single
# directory, but it nonetheless works fine for the examples given here.
det1_dir = os.path.join(output_dir, 'stage1mod/') # Detector1 pipeline outputs will go here
spec2_dir = os.path.join(output_dir, 'stage2mod/') # Spec2 pipeline outputs will go here
spec3_dir = os.path.join(output_dir, 'stage3mod/') # Spec3 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not create them
if not os.path.exists(det1_dir):
    os.makedirs(det1_dir)
if not os.path.exists(spec2_dir):
    os.makedirs(spec2_dir)
if not os.path.exists(spec3_dir):
    os.makedirs(spec3_dir)

In [31]:
# Output subdirectories to keep background data products organized
det1_bgdir = os.path.join(output_bgdir, 'stage1mod/') # Detector1 pipeline outputs will go here
spec2_bgdir = os.path.join(output_bgdir, 'stage2mod/') # Spec2 pipeline outputs will go here
spec3_bgdir = os.path.join(output_bgdir, 'stage3mod/') # Spec2 pipeline outputs will go here

# We need to check that the desired output directories exist, and if not create them
if (output_bgdir != ''):
    if not os.path.exists(det1_bgdir):
        os.makedirs(det1_bgdir)
    if not os.path.exists(spec2_bgdir):
        os.makedirs(spec2_bgdir)
    if not os.path.exists(spec3_bgdir):
        os.makedirs(spec3_bgdir)

In [8]:
# Start a timer to keep track of runtime
time0 = time.perf_counter()

<hr style="border:1px solid gray"> </hr>

3.<font color='white'>-</font>Detector1 Pipeline <a class="anchor" id="det1"></a>
------------------

<div class="alert alert-block alert-warning">
In this section we process our data through the Detector1 pipeline to create Lvl2a data products (i.e., uncalibrated slope images).
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html
</div>

In [9]:
# First we'll define a function that will call the detector1 pipeline with our desired set of parameters
# We won't enumerate the individual steps
def rundet1(filename, outdir):
    print(filename)
    if os.path.exists(os.path.join(outdir, os.path.basename(filename).replace('_uncal.fits', '_rate.fits'))) and \
        os.path.exists(os.path.join(outdir, os.path.basename(filename).replace('_uncal.fits', '_rateints.fits'))):
        print('Output already exists! Skipping...')
        return

    det1 = Detector1Pipeline() # Instantiate the pipeline
    det1.output_dir = outdir # Specify where the output should go
    
    # Overrides for whether or not certain steps should be skipped
    #det1.dq_init.skip = False
    #det1.saturation.skip = False
    #det1.firstframe.skip = False
    #det1.lastframe.skip = False
    #det1.reset.skip = False
    #det1.linearity.skip = False
    #det1.rscd.skip = False
    #det1.dark_current.skip = False
    #det1.refpix.skip = False
    #det1.jump.skip = False
    #det1.ramp_fit.skip = False
    #det1.gain_scale.skip = False
    # det1.ipc.skip = False
    
    # The jump and ramp fitting steps can benefit from multi-core processing, but this is off by default
    # Turn them on here if desired by choosing how many cores to use (quarter, half, or all)
    det1.jump.maximum_cores='half'
    # det1.jump.find_showers=True
    # det1.jump.extend_snr_threshold=1.2  # (default = 1.2)
    # det1.jump.sat_required_snowball=False
    det1.ramp_fit.maximum_cores='half'
    # This next parameter helps with very bright objects and/or very short ramps
    # det1.jump.three_group_rejection_threshold=100

    # JUMP overrides. 
    # Currently pipeline is not flagging lower-level jumps
    # like we might want it to, so lower thresholds for more
    # aggressive flagging.
    #det1.jump.save_results = True
    det1.jump.rejection_threshold = 3.5         # default 4.0sigma
    det1.jump.min_jump_to_flag_neighbors = 8.0  # default 10sigma

    # Additional JUMP overrides related to CR shower flagging. See
    # JWST pipeline documentation page for details of parameters.
    # https://jwst-pipeline.readthedocs.io/en/stable/jwst/jump/index.html
    det1.jump.expand_large_events   = True   # Turn on shower flagging
    det1.jump.use_ellipses          = True   # True for MIRI; approximate showers as elliptical
    det1.jump.min_jump_area         = 12     # Min # of contiguous pixels to trigger expanded flagging
    det1.jump.sat_required_snowball = False  # Do not require pixels to be saturated to flag
    det1.jump.expand_factor         = 3.0    # expands showers beyond ID'd jumps; default 2.0
    
    det1.jump.after_jump_flag_dn1   = 10     # These 4 related to how long after a jump is identified
    det1.jump.after_jump_flag_time1 = 20     #  we should keep flagging the following integrations
    det1.jump.after_jump_flag_dn2   = 1000
    det1.jump.after_jump_flag_time2 = 3000
    
    # Bad pixel mask overrides
    #det1.dq_init.override_mask = 'myfile.fits'

    # Saturation overrides
    #et1.saturation.override_saturation = 'myfile.fits'
    
    # Reset overrides
    #det1.reset.override_reset = 'myfile.fits'
        
    # Linearity overrides
    #det1.linearity.override_linearity = 'myfile.fits'

    # RSCD overrides
    #det1.rscd.override_rscd = 'myfile.fits'
        
    # DARK overrides
    #det1.dark_current.override_dark = 'myfile.fits'
        
    # GAIN overrides
    #det1.jump.override_gain = 'myfile.fits'
    #det1.ramp_fit.override_gain = 'myfile.fits'
                
    # READNOISE overrides
    #det1.jump.override_readnoise = 'myfile.fits'
    #det1.ramp_fit.override_readnoise = 'myfile.fits'
        
    det1.save_results = True # Save the final resulting _rate.fits files
    det1(filename) # Run the pipeline on an input list of files

In [ ]:
# Now let's look for input files of the form *uncal.fits from the science observation
sstring = input_dir + 'jw*mirifu*uncal.fits'
lvl1b_files = np.array(sorted(glob.glob(sstring)))

# If desired, check that these are the band/channel to use
if ((use_ch != '')&(use_band != '')):
    keep=np.zeros(len(lvl1b_files))
    for ii in range(0,len(lvl1b_files)):
        hdu=fits.open(lvl1b_files[ii])
        hdr=hdu[0].header
        if ((hdr['CHANNEL'] == use_ch)&(hdr['BAND'] == use_band)):
            keep[ii]=1
        hdu.close()
    indx=np.where(keep == 1)
    lvl1b_files=lvl1b_files[indx]

print('Found ' + str(len(lvl1b_files)) + ' science input files to process')

In [ ]:
# Run the pipeline on these input files by a simple loop over our pipeline function
if dodet1:
    for file in lvl1b_files:
        rundet1(file, det1_dir)
else:
    print('Skipping Detector1 processing')

In [ ]:
# Now let's look for input files of the form *uncal.fits from the background observation
sstring = input_bgdir + 'jw*mirifu*uncal.fits'
lvl1b_files = np.array(sorted(glob.glob(sstring)))

# If desired, check that these are the band/channel to use
if ((use_ch != '')&(use_band != '')):
    keep=np.zeros(len(lvl1b_files))
    for ii in range(0,len(lvl1b_files)):
        hdu=fits.open(lvl1b_files[ii])
        hdr=hdu[0].header
        if ((hdr['CHANNEL'] == use_ch)&(hdr['BAND'] == use_band)):
            keep[ii]=1
        hdu.close()
    indx=np.where(keep == 1)
    lvl1b_files=lvl1b_files[indx]

print('Found ' + str(len(lvl1b_files)) + ' background input files to process')

In [ ]:
# Run the pipeline on these input files by a simple loop over our pipeline function
if dodet1bg:
    for file in lvl1b_files:
        rundet1(file, det1_bgdir)
else:
    print('Skipping Detector1 processing')

In [ ]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

<hr style="border:1px solid gray"> </hr>

In [ ]:
# Bad pixel mask refinement. Apologies that this code is not the most
# well-organized or prettiest....
dobadpix = True

if dobadpix:
    # This is a long command that sets... wait for it... dnubit = 1.
    dnubit = dqflags.interpret_bit_flags('DO_NOT_USE',mnemonic_map=datamodels.dqflags.pixel)
    
    # We will start with the LONG detector. Grab the background exposures.
    longrates = sorted(glob.glob(det1_bgdir+'*mirifulong*rate.fits'))
    nlong = len(longrates)
    
    # Assemble background exposures
    longstack = np.zeros([nlong,1024,1032])
    for i,exposure in enumerate(longrates):
        hdu = fits.open(exposure)
        longstack[i,:,:] = hdu['SCI'].data
        hdu.close()
    
    # Take the median of the exposures, then fix any pixels where the median is nan
    longmedian = np.median(longstack,axis=0)
    fillvalue  = np.nanmedian(longmedian)
    print('Long stack fill value = {0:.3f}'.format(fillvalue))
    
    dqlong = (fits.open(longrates[0]))['DQ'].data
    bad = np.where((dqlong & dnubit) != 0)
    longmedian[bad] = fillvalue
    
    # Now write our median'd background to disk.
    hdu = fits.PrimaryHDU(longmedian)
    hdu.writeto(det1_bgdir+'background_median_mirifulong.fits',overwrite=True)
    
    
    # And the SHORT detector
    shortrates = sorted(glob.glob(det1_bgdir+'*mirifushort*rate.fits'))
    nshort = len(shortrates)
    
    # Assemble background exposures
    shortstack = np.zeros([nshort,1024,1032])
    for i,exposure in enumerate(shortrates):
        hdu = fits.open(exposure)
        shortstack[i,:,:] = hdu['SCI'].data
        hdu.close()
    
    # Take the median of the exposures, then fix any pixels where the median is nan
    shortmedian = np.median(shortstack,axis=0)
    fillvalue  = np.nanmedian(shortmedian)
    print('Short stack fill value = {0:.3f}'.format(fillvalue))
    
    dqshort = (fits.open(shortrates[0]))['DQ'].data
    bad = np.where((dqshort & dnubit) != 0)
    shortmedian[bad] = fillvalue
    
    # Now write our median'd background to disk.
    hdu = fits.PrimaryHDU(shortmedian)
    hdu.writeto(det1_bgdir+'background_median_mirifushort.fits',overwrite=True)    
    

    # Now we have to identify the bad pixels based on their percentile value, but
    # since each channel has different pixel distributions (e.g. Ch4 has very low
    # throughput), we need to ID which pixels belong to which channel. This uses
    # the distortion files from the miricoord package.

    # Start again with the long exposures...
    # Note in this folder
    # MIRIFULONG vs MIRIFUSHORT = the two different detectors
    # 12SHORT / 12MEDIUM / 12LONG = channels 1 & 2 for band A=short B=medium C=long
    # 34SHORT / 34MEDIUM / 34LONG = channels 3 & 4 for band A=short B=medium C=long
    # so really having "34" and MIRIFULONG in the names is redundant, but whatever...
    miricoordpath = '/'.join(miricoord.__file__.split('/')[:-2])
    longslicemask  = miricoordpath + '/data/fits/flt3/MIRI_FM_MIRIFULONG_34LONG_DISTORTION_9B.05.07.fits'
    longslicemask  = (fits.open(longslicemask))['SLICE_NUMBER'].data[0,:,:]
    shortslicemask = miricoordpath + '/data/fits/flt3/MIRI_FM_MIRIFUSHORT_12LONG_DISTORTION_9B.05.07.fits'
    shortslicemask = (fits.open(shortslicemask))['SLICE_NUMBER'].data[0,:,:]
    ch1 = (shortslicemask >= 100) & (shortslicemask <= 130)
    ch2 = (shortslicemask >= 200) & (shortslicemask <= 230)
    ch3 = (longslicemask >= 300) & (longslicemask <= 330)
    ch4 = (longslicemask >= 400) & (longslicemask <= 430)
    
    # Get wavelength map for channel 4C. Since throughput falls so rapidly
    # towards the longer Ch4 wavelengths, we will calculate the background median
    # in wavelength chunks. Ch3 is way less affected by this so we just do it in
    # one fell swoop.
    wave4c = mrstools.waveimage('4C')
    
    # Subtract Ch3 average from Ch3 pixels
    ch3pix = np.where(ch3)
    ch3med = np.nanmedian(longmedian[ch3pix])
    longmedian[ch3pix] = longmedian[ch3pix] - ch3med
    print('For Ch3, median value was {0:.3f}'.format(ch3med))
    
    # The background in Ch4 varies strongly with wavelength so we have to subdivide it.
    # Ch4C spans from 23.942 - 28.785 um (unclear why this is different than the 
    #  jdox page on MRS, 23.22 - 28.10 um.....). 
    # Let's split it into ~0.33um spans, subtract a wave-dependent background.
    #mins = [ 1.0, 24.5, 25.0, 25.5, 26.0, 26.5, 27.0, 27.5, 28.0]
    mins = [1.0, 24.33, 24.67, 25.0, 25.33, 25.67, 26.0, 26.33, 26.67, 27.0, 27.33, 27.67, 28.0, 28.33]
    maxs = mins[1:] + [50.0]
    for i,lowave in enumerate(mins):
        hiwave = maxs[i]
        ch4pix = np.where(ch4 & (wave4c > lowave) & (wave4c <= hiwave))
        ch4med = np.nanmedian(longmedian[ch4pix])
        longmedian[ch4pix] = longmedian[ch4pix] - ch4med
        print('For wavelength range {0:.1f} - {1:.1f}um, found {2:.0f} pixels.'.format(lowave,hiwave,ch4pix[0].size))
        print('  Median value was {0:.3f}'.format(ch4med))
    
    # Update the median image with a new fill value, write to disk again...
    bad = np.where((dqlong & dnubit) != 0)
    fillvalue = np.nanmedian(longmedian)
    longmedian[bad] = fillvalue
    print('Long stack re-fill value = {0:.3f}'.format(fillvalue))
    hdu = fits.PrimaryHDU(longmedian)
    hdu.writeto(det1_bgdir+'background_median_refill_mirifulong.fits',overwrite=True)    
    
    # Now we start ID'ing pixels to mask, starting with channel 3
    longdqflags = np.zeros_like(dqlong)
    
    good = np.where(ch3 & (longdqflags == 0) & (np.isfinite(longmedian)))
    f = plt.figure()
    ax = f.add_subplot(111)
    ax.plot(longmedian[good],'.',ms=0.5)
    ax.set_ylim(-1,1)
    ax.set_title('Channel 3 pixel values')
    
    # these values were identified 'by hand'; an alternative would be to
    # make some kind of percentile-based or sigma-clipped threshold value
    ch3lowcut, ch3hicut = -0.16, +0.10
    
    ax.axhline(ch3lowcut,color='k',ls='--')
    ax.axhline(ch3hicut, color='k',ls='--')
    plt.show()
    plt.close(f)
    
    tomask = np.where(ch3 & (longmedian < ch3lowcut))
    longdqflags[tomask] = 1
    tomask = np.where(ch3 & (longmedian > ch3hicut))
    longdqflags[tomask] = 1
    
    # Now do the same thing for Ch4
    good = np.where(ch4 & (longdqflags == 0) & (np.isfinite(longmedian)))
    f = plt.figure()
    ax = f.add_subplot(111)
    ax.plot(longmedian[good],'.',ms=0.5)
    ax.set_ylim(-1,1)
    ax.set_title('Channel 4 pixel values')
    
    # again ID'd cutoffs by hand from plot above
    ch4lowcut, ch4hicut = -0.16, +0.27
    
    ax.axhline(ch4lowcut,color='k',ls='--')
    ax.axhline(ch4hicut, color='k',ls='--')
    plt.show()
    plt.close(f)
    
    tomask = np.where(ch4 & (longmedian < ch4lowcut))
    longdqflags[tomask] = 1
    tomask = np.where(ch4 & (longmedian > ch4hicut))
    longdqflags[tomask] = 1
    
    longmedian_clean = longmedian.copy()
    bad = np.where(longdqflags == 1)
    longmedian_clean[bad] = fillvalue
    
    # Save the final 'clean' background to disk for long-wave detector
    hdu = fits.PrimaryHDU(longmedian_clean)
    hdu.writeto(det1_bgdir+'background_median_clean_mirifulong.fits',overwrite=True)
    
    # Now apply correction to our bg + sci files.
    longrates = sorted(glob.glob(det1_dir+'jw*mirifulong_rate.fits'))
    for i,rate in enumerate(longrates):
        hdu = fits.open(rate)
        outfile = rate.replace('rate','ratemod')
        dq = hdu['DQ'].data
        dq[bad] = np.bitwise_or(dq[bad], dnubit)
        hdu['DQ'].data = dq
        hdu.writeto(outfile,overwrite=True)
    longrates = sorted(glob.glob(det1_bgdir+'jw*mirifulong_rate.fits'))
    for i,rate in enumerate(longrates):
        hdu = fits.open(rate)
        outfile = rate.replace('rate','ratemod')
        dq = hdu['DQ'].data
        dq[bad] = np.bitwise_or(dq[bad], dnubit)
        hdu['DQ'].data = dq
        hdu.writeto(outfile,overwrite=True)

    # Now we repeat the above, but for Ch1 & 2 on the short-wave detector
    # Because the background in Chs 1 & 2 is much lower and more uniform
    # than 3&4, we don't need to process them separately and can just ID
    # one set of bad pixel threshold cuts.
    
    # Subtract Ch1 average from Ch1 pixels
    ch1pix = np.where(ch1)
    ch1med = np.nanmedian(shortmedian[ch1pix])
    shortmedian[ch1pix] = shortmedian[ch1pix] - ch1med
    print('For Ch1, median value was {0:.3f}'.format(ch1med))
    
    # Subtract Ch2 average from Ch2 pixels
    ch2pix = np.where(ch2)
    ch2med = np.nanmedian(shortmedian[ch2pix])
    shortmedian[ch2pix] = shortmedian[ch2pix] - ch2med
    print('For Ch2, median value was {0:.3f}'.format(ch2med))
    
    # Update the median image with a new fill value, write to disk again...
    bad = np.where((dqshort & dnubit) != 0)
    fillvalue = np.nanmedian(shortmedian)
    shortmedian[bad] = fillvalue
    print('Short stack re-fill value = {0:.3f}'.format(fillvalue))
    hdu = fits.PrimaryHDU(shortmedian)
    hdu.writeto(det1_bgdir+'background_median_refill_mirifushort.fits',overwrite=True)    
    
    # Now we start ID'ing pixels to mask
    shortdqflags = np.zeros_like(dqshort)
    
    good = np.where((shortdqflags == 0) & (np.isfinite(shortmedian)))
    
    f = plt.figure()
    ax = f.add_subplot(111)
    ax.plot(shortmedian[good],'.',ms=0.5)
    ax.set_ylim(-1,1)
    ax.set_title('Channels 1&2 pixel values')

    lowcut, hicut = -0.14, +0.13
    
    ax.axhline(lowcut,color='k',ls='--')
    ax.axhline(hicut, color='k',ls='--')
    plt.show()
    plt.close(f)
    
    tomask = np.where((shortmedian < lowcut))
    shortdqflags[tomask] = 1
    tomask = np.where((longmedian > hicut))
    shortdqflags[tomask] = 1

    shortmedian_clean = shortmedian.copy()
    bad = np.where(shortdqflags == 1)
    shortmedian_clean[bad] = fillvalue
    
    hdu = fits.PrimaryHDU(shortmedian_clean)
    hdu.writeto(det1_bgdir+'background_median_clean_mirifushort.fits',overwrite=True)
    
    # Now apply correction to our bg + sci files.
    shortrates = sorted(glob.glob(det1_dir+'jw*mirifushort_rate.fits'))
    for i,rate in enumerate(shortrates):
        hdu = fits.open(rate)
        outfile = rate.replace('rate','ratemod')
        dq = hdu['DQ'].data
        dq[bad] = np.bitwise_or(dq[bad], dnubit)
        hdu['DQ'].data = dq
        hdu.writeto(outfile,overwrite=True)
    shortrates = sorted(glob.glob(det1_bgdir+'jw*mirifushort_rate.fits'))
    for i,rate in enumerate(shortrates):
        hdu = fits.open(rate)
        outfile = rate.replace('rate','ratemod')
        dq = hdu['DQ'].data
        dq[bad] = np.bitwise_or(dq[bad], dnubit)
        hdu['DQ'].data = dq
        hdu.writeto(outfile,overwrite=True)


4.<font color='white'>-</font>Spec2 Pipeline <a class="anchor" id="spec2"></a>
------------------

<div class="alert alert-block alert-warning">
In this section we process our data through the Spec2 pipeline in order to produce Lvl2b data products (i.e., calibrated slope images and quick-look data cubes and 1d spectra).  
See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html
</div>

In [20]:
# Define a useful function to write out a Lvl2 association file from an input list
def writel2asn(scifiles, bgfiles, asnfile, prodname):
    # Define the basic association of science files
    asn = afl.asn_from_list(scifiles, rule=DMSLevel2bBase, product_name=prodname)
        
    # Add background files to the association
    nbg=len(bgfiles)
    for ii in range(0,nbg):
        asn['products'][0]['members'].append({'expname': bgfiles[ii], 'exptype': 'background'})
        
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)

In [21]:
# Define a function that will call the spec2 pipeline with our desired set of parameters
# We'll list the individual steps just to make it clear what's running
def runspec2(filename, outdir, nocubes=False):
    spec2 = Spec2Pipeline()
    spec2.output_dir = outdir

    # Assign_wcs overrides
    #spec2.assign_wcs.override_distortion = 'myfile.asdf'
    #spec2.assign_wcs.override_regions = 'myfile.asdf'
    #spec2.assign_wcs.override_specwcs = 'myfile.asdf'
    #spec2.assign_wcs.override_wavelengthrange = 'myfile.asdf'

    # Flatfield overrides
    #spec2.flat.override_flat = 'myfile.fits'
        
    # Straylight overrides
    #spec2.straylight.override_mrsxartcorr = 'myfile.fits'
        
    # Fringe overrides
    #spec2.fringe.override_fringe = 'myfile.fits'
    
    # Photom overrides
    #spec2.photom.override_photom = 'myfile.fits'

    # Cubepar overrides
    #spec2.cube_build.override_cubepar = 'myfile.fits'
        
    # Extract1D overrides
    #spec2.extract_1d.override_extract1d = 'myfile.asdf'
    #spec2.extract_1d.override_apcorr = 'myfile.asdf'
        
    # Overrides for whether or not certain steps should be skipped
    #spec2.assign_wcs.skip = False
    spec2.bkg_subtract.skip = False   # 2D background subtraction!
    spec2.bkg_subtract.save_combined_background = True
    #spec2.flat_field.skip = False
    #spec2.srctype.skip = False
    #spec2.straylight.skip = False
    #spec2.fringe.skip = False
    #spec2.photom.skip = False
    spec2.residual_fringe.skip = False
    #spec2.cube_build.skip = False
    #spec2.extract_1d.skip = False

    # Run pixel replacement code to extrapolate values for otherwise bad pixels.
    # This can help mitigate small 5-10% negative dips in spectra of bright sources.
    spec2.pixel_replace.skip = False
    spec2.pixel_replace.algorithm = 'mingrad'
    
    # This nocubes option allows us to skip the cube building and 1d spectral extraction for individual
    # science data frames, but run it for the background data (as the 1d spectra are needed later
    # for the master background step in Spec3)
    if (nocubes):
        spec2.cube_build.skip = True
        spec2.extract_1d.skip = True
    
    # Some cube building options
    spec2.cube_build.weighting='drizzle'
    spec2.cube_build.coord_system='ifualign' # If aligning cubes with IFU axes instead of sky
      
    spec2.save_results = True
    spec2(filename)

In [ ]:
# Look for uncalibrated science slope files from the Detector1 pipeline
sstring = det1_dir + 'jw*mirifu*ratemod.fits'
ratefiles = np.array(sorted(glob.glob(sstring)))

# If desired, check that these are the band/channel to use
if ((use_ch != '')&(use_band != '')):
    keep=np.zeros(len(ratefiles))
    for ii in range(0,len(ratefiles)):
        hdu=fits.open(ratefiles[ii])
        hdr=hdu[0].header
        if ((hdr['CHANNEL'] == use_ch)&(hdr['BAND'] == use_band)):
            keep[ii]=1
        hdu.close()
    indx=np.where(keep == 1)
    ratefiles=ratefiles[indx]

print('Found ' + str(len(ratefiles)) + ' input files to process')

In [ ]:
# Look for uncalibrated background slope files from the Detector1 pipeline
sstring = det1_bgdir + 'jw*mirifu*ratemod.fits'
bg_ratefiles = np.array(sorted(glob.glob(sstring)))

# If desired, check that these are the band/channel to use
if ((use_ch != '')&(use_band != '')):
    keep=np.zeros(len(bg_ratefiles))
    for ii in range(0,len(bg_ratefiles)):
        hdu=fits.open(bg_ratefiles[ii])
        hdr=hdu[0].header
        if ((hdr['CHANNEL'] == use_ch)&(hdr['BAND'] == use_band)):
            keep[ii]=1
        hdu.close()
    indx=np.where(keep == 1)
    bg_ratefiles=bg_ratefiles[indx]

print('Found ' + str(len(bg_ratefiles)) + ' input files to process')

In [ ]:
if dospec2:
    for (i, file) in enumerate(ratefiles):
        # Make an association file that includes all of the different exposures
        asnfile=os.path.join(output_dir, f'l2asn_{i}.json')
                
        # Look for uncalibrated background slope files from the Detector1 pipeline
        sstring = det1_bgdir + 'jw*mirifu*rate.fits'
        bg_ratefiles = np.array(sorted(glob.glob(sstring)))

        hdu = fits.open(file)
        hdr = hdu[0].header
        use_ch_i = hdr['CHANNEL']
        use_bd_i = hdr['BAND']

        # If desired, check that these are the band/channel to use
        keep=np.zeros(len(bg_ratefiles))
        for ii in range(0,len(bg_ratefiles)):
            hdu=fits.open(bg_ratefiles[ii])
            hdr=hdu[0].header
            if ((hdr['CHANNEL'] == use_ch_i)&(hdr['BAND'] == use_bd_i)):
                keep[ii]=1
            hdu.close()
        indx=np.where(keep == 1)
        bg_ratefiles=bg_ratefiles[indx]

        writel2asn([file], bg_ratefiles, asnfile, 'Level2b')
        runspec2(asnfile, spec2_dir, nocubes=True)
else:
    print('Skipping Spec2 processing')

In [ ]:
sstring = det1_bgdir + 'jw*mirifu*rate.fits'
bg_ratefiles = np.array(sorted(glob.glob(sstring)))

if dospec2bg:
    for file in bg_ratefiles:
        runspec2(file, spec2_bgdir)
else:
    print('Skipping Spec2 processing')

In [ ]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

<hr style="border:1px solid gray"> </hr>

5.<font color='white'>-</font>Spec3 Pipeline: Default configuration (4 per-channel cubes)<a class="anchor" id="spec3"></a>
------------------

<div class="alert alert-block alert-warning">
Here we'll run the Spec3 pipeline to produce a composite data cube from all dithered exposures.
We will need to create an association file from all science and background data in order for the pipeline to use them appropriately.

A word of caution: the data cubes created by the JWST pipeline are in SURFACE BRIGHTNESS units (MJy/steradian), not flux units.  What that means is that if you intend to sum spectra within an aperture you need to be sure to multiply by the pixel area in steradians first in order to get a spectrum in flux units (the PIXAR_SR keyword can be found in the SCI extension header).  This correction is already build into the pipeline Extract1D algorithm.

See https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec3.html
    
</div>

In [24]:
# Define a useful function to write out a Lvl3 association file from an input list
# Note that any background exposures have to be of type x1d.
def writel3asn(scifiles, bgfiles, asnfile, prodname):
    # Define the basic association of science files
    asn = afl.asn_from_list(scifiles, rule=DMS_Level3_Base, product_name=prodname)
        
    # Add background files to the association
    nbg=len(bgfiles)
    for ii in range(0,nbg):
        asn['products'][0]['members'].append({'expname': bgfiles[ii], 'exptype': 'background'})
        
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)

In [ ]:
# Find and sort all of the input files

# Science Files need the cal.fits files
sstring = spec2_dir + 'jw*mirifu*cal.fits'
calfiles = np.array(sorted(glob.glob(sstring)))

# If desired, check that these are the band/channel to use
if ((use_ch != '')&(use_band != '')):
    keep=np.zeros(len(calfiles))
    for ii in range(0,len(calfiles)):
        hdu=fits.open(calfiles[ii])
        hdr=hdu[0].header
        if ((hdr['CHANNEL'] == use_ch)&(hdr['BAND'] == use_band)):
            keep[ii]=1
        hdu.close()
    indx=np.where(keep == 1)
    calfiles=calfiles[indx]

# Background Files need the x1d.fits files
sstring = spec2_bgdir + 'jw*mirifu*x1d.fits'
# sstring = spec2_bgdir + 'jw*mirifu*cal.fits'
bgfiles = np.array(sorted(glob.glob(sstring)))

print('Found ' + str(len(calfiles)) + ' science files to process')
print('Found ' + str(len(bgfiles)) + ' background files to process')

In [ ]:
calfiles_dither1 = [cf for cf in calfiles if "00001" in cf]
calfiles_dither2 = [cf for cf in calfiles if "00002" in cf]
calfiles_dither3 = [cf for cf in calfiles if "00003" in cf]
calfiles_dither4 = [cf for cf in calfiles if "00004" in cf]
print(calfiles)

In [ ]:
# Make an association file that includes all of the different exposures
asnfile=os.path.join(output_dir, 'l3asn.json')
if dospec3:
    writel3asn(calfiles, bgfiles, asnfile, 'Level3')

In [40]:
# This is designed to run on an association file
def runspec3(filename):
    # This initial setup is just to make sure that we get the latest parameter reference files
    # pulled in for our files.  This is a temporary workaround to get around an issue with
    # how this pipeline calling method works.
    crds_config = Spec3Pipeline.get_config_from_reference(filename)
    crds_config['steps']['extract_1d'].pop('ifu_covar_scale')   # ?????????????????????????
    spec3 = Spec3Pipeline.from_config_section(crds_config)
    
    spec3.output_dir = spec3_dir
    spec3.save_results = True
    
    # Cube building configuration options
    # spec3.cube_build.output_file = 'mycube' # Custom output name
    spec3.cube_build.output_type = 'band' # 'band', 'channel', or 'multi' type cube output
    #spec3.cube_build.channel = '1' # Build everything from just channel 1 into a single cube (we could also choose '2','3','4', or 'ALL')
    spec3.cube_build.weighting = 'drizzle' # 'emsm' or 'drizzle'
    spec3.cube_build.coord_system = 'ifualign' # 'ifualign', 'skyalign', or 'internal_cal'
    #spec3.cube_build.scalexy = 0.5 # Output cube spaxel scale (arcsec) if setting it by hand
    #spec3.cube_build.scalew = 0.002 # Output cube voxel depth in wavelength if setting it by hand
    #spec3.cube_build.ra_center = 65.0 # Force cube to be centered at this R.A.
    #spec3.cube_build.dec_center = -35.0 # Force cube to be centered at this Decl.
    #spec3.cube_build.cube_pa = 45.0 # Force cube to have this position angle
    #spec3.cube_build.nspax_x = 61 # Require this number of spaxels in cube X direction
    #spec3.cube_build.nspax_y = 61 # Require this number of spaxels in cube Y direction
    #spec3.cube_build.wavemin = 4.8 # Custom minimum wavelength for the cube
    #spec3.cube_build.wavemax = 6.3 # Custom maximum wavelength for the cube

    # Overrides for whether or not certain steps should be skipped
    #spec3.assign_mtwcs.skip = False

    # spec3.master_background.skip = False

    spec3.mrs_imatch.skip = True
    spec3.outlier_detection.skip = False
    spec3.outlier_detection.kernel_size = '7 7'
    spec3.outlier_detection.threshold_percent = 95.0
    spec3.outlier_detection.grow = 3
    
    spec3.cube_build.skip = False
    spec3.extract_1d.skip = True
    
    # Cubepar overrides
    #spec3.cube_build.override_cubepar = 'myfile.fits'

    # Extract1D overrides and config options
    #spec3.extract_1d.override_extract1d = 'myfile.asdf'
    #spec3.extract_1d.override_apcorr = 'myfile.asdf'
    spec3.extract_1d.ifu_autocen = True # Enable auto-centering of the extraction aperture
    #spec3.extract_1d.center_xy=(20,20) # Override aperture location if desired
    spec3.extract_1d.ifu_rfcorr = True # Turn on 1d residual fringe correction


    spec3(filename)

In [ ]:
if dospec3:
    runspec3(asnfile)
else:
    print('Skipping Spec3 processing')

In [ ]:
# Print out the time benchmark
time1 = time.perf_counter()
print(f"Runtime so far: {time1 - time0:0.4f} seconds")

In [ ]:
from photutils import CircularAperture, CircularAnnulus, SkyCircularAperture, SkyCircularAnnulus, aperture_photometry
from photutils.background import Background2D, MedianBackground
from astropy.wcs import WCS

dopost = True
poststage3 = os.path.join(output_dir, 'poststage3/')
if not os.path.exists(poststage3):
    os.mkdir(poststage3)

ch3med = fits.open(spec3_dir+'Level3_ch3-medium_s3d.fits')
scihead = ch3med['SCI'].header
wcs3med = WCS(ch3med['SCI'].header, naxis=2)
scale3med = np.sqrt(ch3med['SCI'].header['PIXAR_A2'])
wave3 = (np.arange(scihead['NAXIS3']) + scihead['CRPIX3'] - 1) * scihead['CDELT3'] + scihead['CRVAL3']
wave3med = np.median(wave3)

for channel in (1,2,3,4):
    for band in ('short', 'medium', 'long'):
        # Read in the pipe-produced Level 3 data cube for channel 3
        cube = fits.open(spec3_dir+f'Level3_ch{channel}-{band}_s3d.fits')
        sci  = cube['SCI'].data
        err  = cube['ERR'].data
        dq   = cube['DQ'].data
        scihead = cube['SCI'].header
        wcs = WCS(scihead, naxis=2)
        wave = (np.arange(scihead['NAXIS3']) + scihead['CRPIX3'] - 1) * scihead['CDELT3'] + scihead['CRVAL3']
        wavemed = np.median(wave)

        # These will be useful
        wts  = err**-2   # w = 1/sigma^2, useful for weighted averages etc.
        scimasked = np.ma.masked_array(sci, mask = (dq>0)) # use DQ array to mask the science data
        
        # These define the pixel coordinates for the x and y center of the foreground lens,
        # plus a radius based on the Einstein ring size. I (JS) get consistent results whether
        # I get these using the cube's WCS information and ancillary ALMA data, or by collapsing
        # the whole cube over wavelength and visibly seeing the lens+ring.

        # Defined for channel 3MED
        xpeak1, ypeak1, rout1 = 18.5, 17.5, 13.0
        # xpeak2, ypeak2, rout2 = 26.5, 30.0, 7.0
        # xpeak3, ypeak3, rout3 = 26.5, 4.0, 7.0
        # xpeak4, ypeak4, rout4 = 14.5, 27.5, 6.0
        # Transform to ch/band
        xpeak1, ypeak1 = wcs.world_to_pixel(wcs3med.pixel_to_world(xpeak1, ypeak1))
        # xpeak2, ypeak2 = wcs.world_to_pixel(wcs3med.pixel_to_world(xpeak2, ypeak2))
        # xpeak3, ypeak3 = wcs.world_to_pixel(wcs3med.pixel_to_world(xpeak3, ypeak3))
        # xpeak4, ypeak4 = wcs.world_to_pixel(wcs3med.pixel_to_world(xpeak4, ypeak4))
        # Also rescale radii
        rout1 *= scale3med / np.sqrt(scihead['PIXAR_A2']) * (0.033 * wavemed + 0.106) / (0.033 * wave3med + 0.106)
        # rout1 = np.maximum(rout1, 2.5 / np.sqrt(scihead['PIXAR_A2']))
        if channel in (1,2):
            rout1 = 2.5 / np.sqrt(scihead['PIXAR_A2'])
            xpeak1 += 3
            ypeak1 -= 3
        if channel == 4:
            rout1 *= 1.2
        # rout2 *= scale3med / np.sqrt(scihead['PIXAR_A2']) * (0.033 * wavemed + 0.106) / (0.033 * wave3med + 0.106)
        # rout3 *= scale3med / np.sqrt(scihead['PIXAR_A2']) * (0.033 * wavemed + 0.106) / (0.033 * wave3med + 0.106)
        # rout4 *= scale3med / np.sqrt(scihead['PIXAR_A2']) * (0.033 * wavemed + 0.106) / (0.033 * wave3med + 0.106)
        
        # Now we create a boolean mask, circular in shape, using the above params.
        # This atrocious command masks a circle with radius=rout pix around the lens center
        circmask1 = (np.linalg.norm(np.argwhere(sci[0]) - np.array([ypeak1,xpeak1]), axis=1) < rout1).reshape(sci[0].shape)
        # circmask2 = (np.linalg.norm(np.argwhere(sci[0]) - np.array([ypeak2,xpeak2]), axis=1) < rout2).reshape(sci[0].shape)
        # circmask3 = (np.linalg.norm(np.argwhere(sci[0]) - np.array([ypeak3,xpeak3]), axis=1) < rout3).reshape(sci[0].shape)
        # circmask4 = (np.linalg.norm(np.argwhere(sci[0]) - np.array([ypeak4,xpeak4]), axis=1) < rout4).reshape(sci[0].shape)
        
        # Now use that (2D) circular mask to mask the source in all (3D) cube channels.
        # Optionally I (JS) explored also masking some of the edge pixels which seem to
        # be junk, but they also have high ERR / low weight so in weighted stats they matter little
        sourcemask    = np.zeros(sci.shape, dtype=bool)
        # sourcemask[:] = circmask1 | circmask2 | circmask3 | circmask4
        sourcemask[:] = circmask1
        if band == 'long' and channel == 3:
            sourcemask[:, :, :5] = True
        # sourcemask[:, :, :2] = True # mask edges
        # sourcemask[:, :, 32] = True
        
        # Create a copy of the original (DQ masked) data but now include our
        # additional mask from above
        scisourcemask = scimasked.copy()
        scisourcemask.mask += sourcemask
        
        # We will calculate the stripe template using a running average over the
        # cube. There's an option to use either a straight average or a weighted average,
        # so we'll just make both.
        # These will be the smoothed cubes before stripe removal
        cubeavg     = np.zeros(scisourcemask.shape)
        cubeavg_wtd = np.zeros(scisourcemask.shape)
        # These will contain the estimated stripe templates
        cubebkg     = np.zeros(scisourcemask.shape)
        cubebkg_wtd = np.zeros(scisourcemask.shape)
        
        # Setup for our background/stripe estimation. Stripes are coherent over tens
        # of wavelength channels (remember cube has been "3d drizzled" so oversamples
        # the detectors spectral resolution). After some experimentation I settled on
        # a 25-channel running average.
        chstep   = 25
        halfstep = int((chstep-1)/2)
        bkg_estimator = MedianBackground() # From photutils
        
        for chstart in np.arange(halfstep, sci.shape[0]):
            cutout = np.ma.average(scimasked[chstart-halfstep:chstart+chstep+halfstep],axis=0)
            cutout2= np.ma.average(scimasked[chstart-halfstep:chstart+chstep+halfstep],axis=0,weights=wts[chstart-halfstep:chstart+chstep+halfstep])

            # Use photutils to estimate the 2D "background" (striping). The box_size
            # parameter sets the shape of the stripe estimation. Here using (1,shape[1]/2)
            # corresponds to fitting the "background" in a shape that is 1 row tall and
            # half the x-pixels of the cube, which allows enough flexibility to account
            # for the fact that the slices aren't perfectly x-aligned due to the slice
            # curvature on the detector. Also exclude_percentile is very high here because
            # in some rows most of the cube pixels are masked, where our circular source
            # mask is at its widest point.
            try:
                bkg = Background2D(cutout,  box_size=(1,int(np.ceil(cutout.shape[1]))), mask=(cutout.mask | sourcemask[0]), 
                        filter_size=1, bkg_estimator=bkg_estimator, exclude_percentile=85.0)
                bkg2= Background2D(cutout2, box_size=(1,int(np.ceil(cutout.shape[1]))), mask=(cutout2.mask | sourcemask[0]), 
                        filter_size=1, bkg_estimator=bkg_estimator, exclude_percentile=85.0)    

                # Overwrite our blank cubes with data
                cubeavg[chstart]     = cutout
                cubeavg_wtd[chstart] = cutout2
                cubebkg[chstart]     = bkg.background
                cubebkg_wtd[chstart] = bkg2.background

            except ValueError:

                cubeavg[chstart]     = cutout
                cubeavg_wtd[chstart] = cutout2
                cubebkg[chstart]     = 0.
                cubebkg_wtd[chstart] = 0.
            
        # The above fails near the edge channels at the start/end of the cube,
        # pad those slices with the same background for convenience (and remember
        # to ignore edge channels in later analysis)
        cubeavg[0:halfstep]     = cubeavg[halfstep]
        cubeavg_wtd[0:halfstep] = cubeavg_wtd[halfstep]
        cubebkg[0:halfstep]     = cubebkg[halfstep]
        cubebkg_wtd[0:halfstep] = cubebkg[halfstep]
        cubeavg[-halfstep:]     = cubeavg[-halfstep]
        cubeavg_wtd[-halfstep:] = cubeavg_wtd[-halfstep]
        cubebkg[-halfstep:]     = cubebkg[-halfstep]
        cubebkg_wtd[-halfstep:] = cubebkg[-halfstep]

        # subtract the median background value so that we're not oversubtracting any real signal
        for i in np.arange(sci.shape[0]):
            cubebkg[i,:,:] = cubebkg[i,:,:] - np.nanmedian(cubebkg[i,:,:])
            cubebkg_wtd[i,:,:] = cubebkg_wtd[i,:,:] - np.nanmedian(cubebkg_wtd[i,:,:])
        
        # Now write the wavelength-smoothed cubes and backgrounds to disk.
        cubehdu = fits.PrimaryHDU(cubeavg,header=cube['SCI'].header)
        cubehdu.writeto(poststage3+f'Level3_ch{channel}-{band}_s3d_runningavg.fits',overwrite=True)
        cubehdu = fits.PrimaryHDU(cubeavg_wtd,header=cube['SCI'].header)
        cubehdu.writeto(poststage3+f'Level3_ch{channel}-{band}_s3d_runningweightedavg.fits',overwrite=True)
        bkghdu = fits.PrimaryHDU(cubebkg,header=cube['SCI'].header)
        bkghdu.writeto(poststage3+f'Level3_ch{channel}-{band}_s3d_background.fits',overwrite=True)
        bkghdu = fits.PrimaryHDU(cubebkg_wtd, header=cube['SCI'].header)
        bkghdu.writeto(poststage3+f'Level3_ch{channel}-{band}_s3d_weightedbackground.fits',overwrite=True)
        
        # Finally, also write out the background-subtracted cubes to disk
        bkgsub     = scimasked.data - cubebkg
        bkgsub_wtd = scimasked.data - cubebkg_wtd
        cubehdu = cube.copy()
        cubehdu['SCI'].data = bkgsub
        cubehdu.writeto(poststage3+f'Level3_ch{channel}-{band}_s3d_bkgsubtracted.fits',overwrite=True)
        cubehdu = cube.copy()
        cubehdu['SCI'].data = bkgsub_wtd
        cubehdu.writeto(poststage3+f'Level3_ch{channel}-{band}_s3d_bkgwtdsubtracted.fits',overwrite=True)

In [ ]:
for channel in (1,2,3,4):
    for band in ('short', 'medium', 'long'):
        from matplotlib.patches import Ellipse

        # The starting and stopping channels of the cubes we should plot
        # You can play with these to see how the stripe properties vary
        # as a function of cube width etc.
        chstart, chstop = 500, 600

        pipe = fits.open(spec3_dir+f'Level3_ch{channel}-{band}_s3d.fits')
        back = fits.open(poststage3+f'Level3_ch{channel}-{band}_s3d_weightedbackground.fits')
        bsub = fits.open(poststage3+f'Level3_ch{channel}-{band}_s3d_bkgwtdsubtracted.fits')

        scihead = pipe['SCI'].header
        wcs = WCS(scihead, naxis=2)
        wave = (np.arange(scihead['NAXIS3']) + scihead['CRPIX3'] - 1) * scihead['CDELT3'] + scihead['CRVAL3']
        wavemed = np.median(wave)

        pipe = pipe['SCI'].data
        back = back['SCI'].data
        bsub = bsub['SCI'].data

        # Defined for channel 3MED
        xpeak1, ypeak1, rout1 = 18.5, 17.5, 13.0
        # xpeak2, ypeak2, rout2 = 25.5, 12.5, 7.0
        # xpeak3, ypeak3, rout3 = 15.5, 20.5, 7.0
        # xpeak4, ypeak4, rout4 = 14.5, 27.5, 6.0
        # Transform to ch/band
        xpeak1, ypeak1 = wcs.world_to_pixel(wcs3med.pixel_to_world(xpeak1, ypeak1))
        # xpeak2, ypeak2 = wcs.world_to_pixel(wcs3med.pixel_to_world(xpeak2, ypeak2))
        # xpeak3, ypeak3 = wcs.world_to_pixel(wcs3med.pixel_to_world(xpeak3, ypeak3))
        # xpeak4, ypeak4 = wcs.world_to_pixel(wcs3med.pixel_to_world(xpeak4, ypeak4))
        # Also rescale radii
        rout1 *= scale3med / np.sqrt(scihead['PIXAR_A2']) * (0.033 * wavemed + 0.106) / (0.033 * wave3med + 0.106)
        # rout1 = np.maximum(rout1, 2.5 / np.sqrt(scihead['PIXAR_A2']))
        if channel in (1,2):
            rout1 = 2.5 / np.sqrt(scihead['PIXAR_A2'])
            xpeak1 += 3
            ypeak1 -= 3
        if channel == 4:
            rout1 *= 1.2
        # rout2 *= scale3med / np.sqrt(scihead['PIXAR_A2'])
        # rout3 *= scale3med / np.sqrt(scihead['PIXAR_A2'])
        # rout4 *= scale3med / np.sqrt(scihead['PIXAR_A2']) * (0.033 * wavemed + 0.106) / (0.033 * wave3med + 0.106)
        
        # Edge pixels are flagged in original cube, but not in mine (we ignored the
        # DQ arrays when writing back to disk), flag them here for consistency
        back[:,:1,:]  = 0.
        back[:,-1:,:] = 0.
        back[:,:,:1]  = 0.
        back[:,:,-1:] = 0.
        bsub[:,:1,:]  = 0.
        bsub[:,-1:,:] = 0.
        bsub[:,:,:2]  = 0.
        bsub[:,:,-2:] = 0.
        
        # Average the cube over the specified channel range
        pipedata = np.ma.average(pipe[chstart:chstop], axis=0)
        backdata = np.ma.average(back[chstart:chstop], axis=0)
        bsubdata = np.ma.average(bsub[chstart:chstop], axis=0)
        
        f,axarr = plt.subplots(figsize=(7.5,3),nrows=1,ncols=3)
        f.subplots_adjust(left=0.01,right=0.99,bottom=0.01,top=0.9,hspace=0,wspace=0)
        
        axarr[0].imshow(pipedata, origin='lower', cmap='jet', vmin=-3., vmax=20.0)
        axarr[1].imshow(backdata, origin='lower', cmap='jet', vmin=-3., vmax=20.0)
        axarr[2].imshow(bsubdata, origin='lower', cmap='jet', vmin=-3., vmax=20.0)
        
        axarr[0].set_title('Original Pipeline',fontsize='large')
        axarr[1].set_title('Estimated Striping',fontsize='large')
        axarr[2].set_title('Stripes Removed',fontsize='large')
        
        for ax in axarr:
            # note we actually shift the drawing of the masked regions bc matplotlib
            # defines pixels differently than numpy (it's a half-pixel thing as always)
            mask = Ellipse(xy=(xpeak1+0.5,ypeak1+0.5),width=2*rout1,height=2*rout1,angle=0.,ec='grey',fc='None')
            ax.add_artist(mask)
            # mask = Ellipse(xy=(xpeak2+0.5,ypeak2+0.5),width=2*rout2,height=2*rout2,angle=0.,ec='grey',fc='None')
            # ax.add_artist(mask)
            # mask = Ellipse(xy=(xpeak3+0.5,ypeak3+0.5),width=2*rout3,height=2*rout3,angle=0.,ec='grey',fc='None')
            # ax.add_artist(mask)
            # mask = Ellipse(xy=(xpeak4+0.5,ypeak4+0.5),width=2*rout4,height=2*rout4,angle=0.,ec='grey',fc='None')
            # ax.add_artist(mask)
            ax.set_xticks([])
            ax.set_yticks([])
        
        plt.show()
        plt.close(f)

<hr style="border:1px solid gray"> </hr>

<img style="float: center;" src="https://www.stsci.edu/~dlaw/stsci_logo.png" alt="stsci_logo" width="200px"/> 